## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..","Weather_Database","WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Khor,47.8861,134.9464,69.28,85,99,1.19,RU,overcast clouds
1,1,Roald,62.5833,6.1333,54.16,93,100,11.50,NO,light rain
2,2,Zhuhai,22.2769,113.5678,96.85,74,40,14.97,CN,scattered clouds
3,3,Asau,46.4333,26.4000,54.52,59,0,4.72,RO,clear sky
4,4,Punta Arenas,-53.1500,-70.9167,30.31,86,75,6.91,CL,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
5,5,Alice Springs,-23.7000,133.8833,71.15,9,1,3.44,AU,clear sky
6,6,Viet Tri,21.3019,105.4308,91.35,71,99,0.43,VN,overcast clouds
7,7,Luba,3.4568,8.5547,73.29,86,92,6.33,GQ,light rain
9,9,Kavaratti,10.5669,72.6420,82.04,79,87,18.19,IN,overcast clouds
12,12,Hilo,19.7297,-155.0900,83.28,94,100,5.99,US,moderate rain
20,20,Albany,42.6001,-73.9662,73.81,79,82,1.01,US,broken clouds
21,21,Matara,5.9485,80.5353,81.91,76,98,10.45,LK,overcast clouds
22,22,Kuah,6.3265,99.8432,87.78,74,20,3.44,MY,few clouds
25,25,Baiao,-2.7906,-49.6717,76.35,74,48,3.36,BR,scattered clouds
27,27,Rikitea,-23.1203,-134.9692,73.42,72,100,14.16,PF,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                362
City                   362
Lat                    362
Lng                    362
Max Temp               362
Humidity               362
Cloudiness             362
Wind Speed             362
Country                361
Current Description    362
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how = 'any')
clean_df.count()

City_ID                361
City                   361
Lat                    361
Lng                    361
Max Temp               361
Humidity               361
Cloudiness             361
Wind Speed             361
Country                361
Current Description    361
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Alice Springs,AU,71.15,clear sky,-23.7000,133.8833,
6,Viet Tri,VN,91.35,overcast clouds,21.3019,105.4308,
7,Luba,GQ,73.29,light rain,3.4568,8.5547,
9,Kavaratti,IN,82.04,overcast clouds,10.5669,72.6420,
12,Hilo,US,83.28,moderate rain,19.7297,-155.0900,
20,Albany,US,73.81,broken clouds,42.6001,-73.9662,
21,Matara,LK,81.91,overcast clouds,5.9485,80.5353,
22,Kuah,MY,87.78,few clouds,6.3265,99.8432,
25,Baiao,BR,76.35,scattered clouds,-2.7906,-49.6717,
27,Rikitea,PF,73.42,overcast clouds,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df.loc[(hotel_df["Hotel Name"] == ""),"Hotel Name"] = np.nan
hotel_df.count()



City                   361
Country                361
Max Temp               361
Current Description    361
Lat                    361
Lng                    361
Hotel Name             339
dtype: int64

In [10]:
clean_hotel_df = hotel_df.dropna(how = 'any')
clean_hotel_df.count()

City                   339
Country                339
Max Temp               339
Current Description    339
Lat                    339
Lng                    339
Hotel Name             339
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))